In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


In [ ]:
f_x30 = open('130_triple/VCS_fields_TT_x_ROM.txt', 'r')
x30 = np.loadtxt(f_x30)
x30 = x30[::100,:]
print(x30.shape)
f_y30 = open('130_triple/VCS_fields_TT_y_ROM.txt', 'r')
y30 = np.loadtxt(f_y30)
y30 = y30[::100,:]
print(y30.shape)
#f_x28 = open('28_single/VCS_fields_TT_x_ROM.txt', 'r')
#x28 = np.loadtxt(f_x28)
#print(x28.shape)
#f_y28 = open('28_single/VCS_fields_TT_y_ROM.txt', 'r')
#y28 = np.loadtxt(f_y28)
#print(y28.shape)
#print(x30[0:5,:])

In [ ]:
dim_x = int(x30.shape[1])
dim_y = int(y30.shape[1])
print(dim_x)
print(dim_y)
#f_sample_grid_d1 = open('sample_grid_d1.txt', 'r')
#sample_grid_d1 = 2*(np.loadtxt(f_sample_grid_d1) - 0.5)
#sample_grid_d1 = np.loadtxt(f_sample_grid_d1)
#print(sample_grid_d1)
#f_sample_grid_d2 = open('sample_grid_d2.txt', 'r')
#sample_grid_d2 = 10*(np.loadtxt(f_sample_grid_d2) - 0.1)
#sample_grid_d2 = np.loadtxt(f_sample_grid_d2)
#print(sample_grid_d2)

In [ ]:
# the sample_grid are the rom_coords including the parameter
# the train_values are the rom_coords at the next step
#
x30_sample_grid_tmp = np.delete(x30, (x30.shape[0]-1), axis=0)
x30_train_values = np.delete(x30, (0), axis=0)
#print(x30)
#print(x30_sample_grid_tmp)
#print(x30_train_values)
x30_sample_grid = 30*np.ones((x30_sample_grid_tmp.shape[0],x30_sample_grid_tmp.shape[1]+1))
x30_sample_grid[:,:-1] = x30_sample_grid_tmp
#print(x30_sample_grid)
print(x30_sample_grid.shape)
print(x30_train_values.shape)

# build the sample and train matrices when going over two timesteps
x30_twostep = np.zeros((x30.shape[0]-1,x30.shape[1]*2))
x30_twostep[:,0:x30.shape[1]] = x30[:-1,:]
x30_twostep[:,x30.shape[1]:2*x30.shape[1]] = x30[1:,:]
print(x30_twostep.shape)
x30_sample_grid_tmp = np.delete(x30_twostep, (x30_twostep.shape[0]-1), axis=0)
#x30_train_values = np.delete(x30_twostep, (0), axis=0)
#x30_sample_grid = 30*np.ones((x30_sample_grid_tmp.shape[0],x30_sample_grid_tmp.shape[1]+1))
#x30_sample_grid[:,:-1] = x30_sample_grid_tmp
print(x30_sample_grid.shape)
print(x30_train_values.shape)
x30_3step = np.zeros((x30.shape[0]-2,x30.shape[1]*3))
x30_3step[:,0:x30.shape[1]] = x30[:-2,:]
x30_3step[:,x30.shape[1]:2*x30.shape[1]] = x30[1:-1,:]
x30_3step[:,2*x30.shape[1]:3*x30.shape[1]] = x30[2:,:]
print(x30_3step.shape)
x30_sample_grid_tmp = np.delete(x30_3step, (x30_3step.shape[0]-1), axis=0)
#x30_train_values = np.delete(x30_3step, (0), axis=0)
#x30_sample_grid = 30*np.ones((x30_sample_grid_tmp.shape[0],x30_sample_grid_tmp.shape[1]+1))
#x30_sample_grid[:,:-1] = x30_sample_grid_tmp
print(x30_sample_grid.shape)
print(x30_train_values.shape)
x30_4step = np.zeros((x30.shape[0]-3,x30.shape[1]*4))
x30_4step[:,0:x30.shape[1]] = x30[:-3,:]
x30_4step[:,x30.shape[1]:2*x30.shape[1]] = x30[1:-2,:]
x30_4step[:,2*x30.shape[1]:3*x30.shape[1]] = x30[2:-1,:]
x30_4step[:,3*x30.shape[1]:4*x30.shape[1]] = x30[3:,:]
x30_sample_grid_tmp = np.delete(x30_4step, (x30_4step.shape[0]-1), axis=0)
x30_train_values = np.delete(x30_4step, (0), axis=0)
x30_sample_grid = 30*np.ones((x30_sample_grid_tmp.shape[0],x30_sample_grid_tmp.shape[1]+1))
x30_sample_grid[:,:-1] = x30_sample_grid_tmp
print(x30_sample_grid.shape)
print(x30_train_values.shape)


In [ ]:
#train_values = samples_optimal_clustering.reshape(110)
#train_values = np.concatenate((lcs0, lcs1, lcs2, lcs3, lcs4, lcs5, lcs6, lcs7))
#print(train_values)
#train_values = np.reshape(train_values,(lcs0.shape[0],8), order='F')
#print(train_values.shape)
#train_values = 1./train_values
#np.argmax(train_values, axis = 1)
#for i in range(0,train_values.shape[0]):
#    sum_row = np.sum(train_values[i,:])
#    train_values[i,:] = train_values[i,:] / sum_row
    #print(sum_row)
#print(train_values)
#train_values = np.transpose(trainANN)
#print(trainANN)
#print(train_values.shape)
#print(train_values.shape[1])
fig, ax = plt.subplots()  # Create a figure containing a single axes.
#ax.plot([1, 2, 3, 4], [1, 4, 2, 3])  # Plot some data on the axes.
ax.plot(x30[:,0])

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=[x30_sample_grid.shape[1]]),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(x30_train_values.shape[1])
])

In [ ]:
es = EarlyStopping(monitor='mse', min_delta = 0, patience = 100, mode='min',verbose=1, restore_best_weights = 1)

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'mse'])
model.fit(x30_sample_grid, x30_train_values, epochs=10000, callbacks=[es])

es = EarlyStopping(monitor='mse', min_delta = 0, patience = 100, mode='min',verbose=1, restore_best_weights = 1)

model.compile(optimizer='adadelta',
              loss='mse',
              metrics=['mae', 'mse'])
model.fit(x30_sample_grid, x30_train_values, epochs=200, callbacks=[es])


In [ ]:
#model.fit(x30_sample_grid, x30_train_values, epochs=10000, callbacks=[es])

In [ ]:
# use the ROM to time-step given the first reduced coordinate representation
current_estimate = x30_sample_grid[0,:]
estimated_trajectory = 0*x30_sample_grid
estimated_trajectory[0,:] = current_estimate
nn = np.r_[current_estimate, 30]
print(current_estimate)
print(x30_sample_grid.shape)
print(nn)
print(x30_sample_grid[0:1,:])
for i in range(0,x30_sample_grid.shape[0]):
    next_estimate = model.predict(estimated_trajectory[i:i+1,:])
    estimated_trajectory[i+1,:-1] = next_estimate
    print(i)
    
#fig, ax = plt.subplots()    
#ax.plot(x30[:,0])
#ax.plot(estimated_trajectory[:,0])

In [ ]:
fig, ax = plt.subplots()    
ax.plot(x30[0:1000,0])
ax.plot(estimated_trajectory[0:1000,0])

In [ ]:
np.argmax(train_values, axis = 1)

In [ ]:
np.argmax(pred_sg, axis=1) - np.argmax(train_values, axis = 1)

In [ ]:
f_fine_sample_grid = open('fine_sample_grid.txt', 'r')
fine_sample_grid = np.loadtxt(f_fine_sample_grid)
print(fine_sample_grid.shape)
print(fine_sample_grid)

In [ ]:
pred_fsg = model.predict(fine_sample_grid)
np.set_printoptions(threshold=np.inf)
print(pred_fsg.shape)
np.savetxt('pred_fsg.txt', pred_fsg)
#np.argmax(pred_fsg, axis=1)

In [ ]:
#f_combination_optimal_c = open('combination_optimal_c.txt', 'r')
#combination_optimal_c = np.loadtxt(f_combination_optimal_c)
#print(combination_optimal_c)
#print(sample_grid.shape)
#print(train_values.shape)
#print(combination_optimal_c.shape)
#detailed_train_values = combination_optimal_c.reshape(1640)
#print(detailed_train_values.shape)

In [ ]:
#model_2 = keras.Sequential([
#    keras.layers.Dense(2, activation='relu'),
#    keras.layers.Dense(1024, activation='relu'),
#    keras.layers.Dense(1024, activation='relu'),
#    keras.layers.Dense(8, activation='softmax')
#])
#model_2.compile(optimizer='adam',
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])
#model_2.fit(fine_sample_grid, detailed_train_values, epochs=500)

In [ ]:
#pred_mod2 = model_2.predict(fine_sample_grid)
#save_pred_mod2_fsg = np.argmax(pred_mod2, axis=1)
#np.savetxt('save_pred_mod2_fsg.txt', save_pred_mod2_fsg)

In [ ]:
pred_mod1 = np.argmax(pred_fsg, axis=1)
#np.savetxt('smart_save_pred_mod1_fsg.txt', pred_mod1)